# Prepare tensorflow-model-optimization for pruning

In [1]:
! pip uninstall -y tensorflow
! pip uninstall -y tf-nightly
! pip install -U tensorflow-gpu==1.14.0

! pip install tensorflow-model-optimization

Uninstalling tensorflow-1.15.0:
  Successfully uninstalled tensorflow-1.15.0
     |████████████████████████████████| 377.0MB 40kB/s 
     |████████████████████████████████| 3.2MB 36.3MB/s 
     |████████████████████████████████| 491kB 40.6MB/s 
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
     |████████████████████████████████| 102kB 3.2MB/s 


In [1]:
%load_ext tensorboard
import tensorboard
import tensorflow as tf
tf.enable_eager_execution()

import tempfile
import zipfile
import os

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

#Download dataset from Google Drive

In [2]:
!pip install PyDrive
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
#https://drive.google.com/open?id=1swPiOWtQ80zWHOK0pyWDuhilhRDkFux8/view?usp=sharing
#https://drive.google.com/open?id=1swPiOWtQ80zWHOK0pyWDuhilhRDkFux8
download = drive.CreateFile({'id': '1swPiOWtQ80zWHOK0pyWDuhilhRDkFux8'})
download.GetContentFile('weed_dataset_w25.hdf5')

# Read Dataset

In [0]:
from sklearn.model_selection import StratifiedKFold
import numpy as np
from operator import truediv
import h5py
from sklearn.decomposition import PCA
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, cohen_kappa_score, confusion_matrix

import tensorboard
import tensorflow as tf
tf.enable_eager_execution()

import tempfile
import zipfile
import os


In [5]:
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
LOAD HDF5 FILE
""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""

hdf5_file = h5py.File('weed_dataset_w25.hdf5', "r")
train_x = np.array(hdf5_file["train_img"][...])
# train_x = train_x / np.max(train_x)
# train_x = np.clip(train_x, 0, 1)
#train_x = np.reshape(train_x, (train_x.shape[0], train_x.shape[1], train_x.shape[2], train_x.shape[3], 1))
train_y = np.array(hdf5_file["train_labels"][...])

# Average consecutive bands
img2 = np.zeros((train_x.shape[0], train_x.shape[1], train_x.shape[2], int(train_x.shape[3]/2)))
for n in range(0, train_x.shape[0]):
    # Average consecutive bands
    for i in range(0, train_x.shape[3], 2):
        img2[n, :, :, int(i/2)] = (train_x[n, :, :, i] + train_x[n, :, :, i + 1]) / 2.

train_x = img2

print(train_x.shape)
train_x = np.reshape(train_x, (train_x.shape[0], train_x.shape[1], train_x.shape[2], train_x.shape[3], 1))

(6316, 25, 25, 150)


# Trained pruned network

In [7]:
data = 'WEED'
loaded_model = tf.keras.models.load_model("weights-hyper3dnet" + data + str(1) + "-best_3layers_4filters.h5")

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [8]:
from tensorflow_model_optimization.sparsity import keras as sparsity

epochs = 8
batch_size = 32;
num_train_samples = train_x.shape[0]
end_step = np.ceil(1.0 * num_train_samples / batch_size).astype(np.int32) * epochs
print(end_step)

new_pruning_params = {
      'pruning_schedule': sparsity.PolynomialDecay(initial_sparsity=0.50,
                                                   final_sparsity=0.90,
                                                   begin_step=0,
                                                   end_step=end_step,
                                                   frequency=100)
}

new_pruned_model = sparsity.prune_low_magnitude(loaded_model, **new_pruning_params)
new_pruned_model.summary()

new_pruned_model.compile(
    loss=tf.keras.losses.categorical_crossentropy,
    optimizer='adadelta',
    metrics=['accuracy'])

1584

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 25, 25, 150, 0                                            
__________________________________________________________________________________________________
prune_low_magnitude_conv3d_7 (P (None, 25, 25, 150,  1018        input_3[0][0]                    
__________________________________________________________________________________________________
prune_low_magnitude_batch_norma (None, 25, 25, 150,  33          prune_low_magnitude_conv3d_7[0][0
__________________________________________________________________________________________________
prune_low_magnitude_activation_ (None, 25, 25, 150,  1           prune_low_magnitude_batch_normali
_______

In [9]:
windowSize = train_x.shape[1]
classes = 3
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
cvoa = []
cvaa = []
cvka = []
cvpre = []
cvrec = []
cvf1 = []
cva1 = []
cva2 = []
cva3 = []

data = 'WEED'
ntrain = 1
for train, test in kfold.split(train_x, train_y):

    ytrain = tf.keras.utils.to_categorical(train_y[train]).astype(np.int32)
    ytest = tf.keras.utils.to_categorical(train_y[test]).astype(np.int32)

    """""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
    """"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
    PRUNING
    """"""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
    """""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
    new_pruning_params = {
          'pruning_schedule': sparsity.PolynomialDecay(initial_sparsity=0.10,
                                                      final_sparsity=0.30,
                                                      begin_step=0,
                                                      end_step=end_step,
                                                      frequency=100)
    }

    loaded_model.load_weights("weights-hyper3dnet" + data + str(ntrain) + "-best_3layers_4filters.h5")

    new_pruned_model = sparsity.prune_low_magnitude(loaded_model, **new_pruning_params)
    #new_pruned_model.summary()

    new_pruned_model.compile(
        loss=tf.keras.losses.categorical_crossentropy,
        optimizer='adadelta',
        metrics=['accuracy'])
    
    # checkpoint
    filepath = "pruned-weights-hyper3dnet" + data + str(ntrain) + "-best_3layers_4filters.h5"
    checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint, sparsity.UpdatePruningStep()]

    # Train model on dataset
    print(data + ": Training" + str(ntrain) + "begins...")
    history = new_pruned_model.fit(x=train_x[train], y=ytrain, validation_data=(train_x[test], ytest),
                        batch_size=32, epochs=epochs, callbacks=callbacks_list)
    
    ntrain += 1


WEED: Training8begins...
Train on 5685 samples, validate on 631 samples
Epoch 1/8
5664/5685 [============================>.] - ETA: 0s - loss: 0.0038 - acc: 0.9993
Epoch 00001: val_acc improved from -inf to 0.98574, saving model to pruned-weights-hyper3dnetWEED8-best_3layers_4filters.h5
5685/5685 [==============================] - 47s 8ms/sample - loss: 0.0038 - acc: 0.9993 - val_loss: 0.0569 - val_acc: 0.9857
Epoch 2/8
5664/5685 [============================>.] - ETA: 0s - loss: 0.0031 - acc: 0.9989
Epoch 00002: val_acc did not improve from 0.98574
5685/5685 [==============================] - 34s 6ms/sample - loss: 0.0031 - acc: 0.9989 - val_loss: 0.1126 - val_acc: 0.9794
Epoch 3/8
5664/5685 [============================>.] - ETA: 0s - loss: 0.0038 - acc: 0.9986
Epoch 00003: val_acc did not improve from 0.98574
5685/5685 [==============================] - 34s 6ms/sample - loss: 0.0038 - acc: 0.9986 - val_loss: 0.0830 - val_acc: 0.9810
Epoch 4/8
5664/5685 [============================>

NameError: ignored

In [13]:
new_pruned_model.load_weights("pruned-weights-hyper3dnet" + data + str(2) + "-best_3layers_4filters.h5")
final_model = sparsity.strip_pruning(new_pruned_model)
final_model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 25, 25, 150, 0                                            
__________________________________________________________________________________________________
conv3d_7 (Conv3D)               (None, 25, 25, 150,  512         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_15 (BatchNo (None, 25, 25, 150,  32          conv3d_7[1][0]                   
__________________________________________________________________________________________________
activation_15 (Activation)      (None, 25, 25, 150,  0           batch_normalization_15[1][0]     
____________________________________________________________________________________________

In [15]:
import numpy as np
for i, w in enumerate(final_model.get_weights()):
    print(
        "{} -- Total:{}, Zeros: {:.4f}%".format(
            final_model.weights[i].name, w.size, np.sum(w == 0) / w.size * 100
        )
    )

conv3d_7/kernel:0 -- Total:504, Zeros: 28.9683%
conv3d_7/bias:0 -- Total:8, Zeros: 0.0000%
batch_normalization_15/gamma:0 -- Total:8, Zeros: 0.0000%
batch_normalization_15/beta:0 -- Total:8, Zeros: 0.0000%
batch_normalization_15/moving_mean:0 -- Total:8, Zeros: 0.0000%
batch_normalization_15/moving_variance:0 -- Total:8, Zeros: 0.0000%
conv3d_8/kernel:0 -- Total:2880, Zeros: 28.9931%
conv3d_8/bias:0 -- Total:8, Zeros: 0.0000%
batch_normalization_16/gamma:0 -- Total:8, Zeros: 0.0000%
batch_normalization_16/beta:0 -- Total:8, Zeros: 0.0000%
batch_normalization_16/moving_mean:0 -- Total:8, Zeros: 0.0000%
batch_normalization_16/moving_variance:0 -- Total:8, Zeros: 0.0000%
conv3d_9/kernel:0 -- Total:3456, Zeros: 28.9931%
conv3d_9/bias:0 -- Total:8, Zeros: 0.0000%
batch_normalization_17/gamma:0 -- Total:8, Zeros: 0.0000%
batch_normalization_17/beta:0 -- Total:8, Zeros: 0.0000%
batch_normalization_17/moving_mean:0 -- Total:8, Zeros: 0.0000%
batch_normalization_17/moving_variance:0 -- Total:8,

In [17]:
print('Saving pruned model to: ', "Kochia_hyper3DNet_pruned.h5")
tf.keras.models.save_model(final_model, "Kochia_hyper3DNet_pruned.h5", 
                        include_optimizer=False)

Saving pruned model to:  Kochia_hyper3DNet_pruned.h5
